In [1]:
#|default_exp showdoc

# showdoc
> Display symbol documentation in notebook and website

In [2]:
#|export
from fastcore.docments import *
from fastcore.utils import *
from importlib import import_module
from nbprocess.doclinks import *
import inspect
from collections import OrderedDict
from dataclasses import dataclass, is_dataclass
from nbprocess.read import get_config

## Rendering docment Tables

Render nicely formatted tables that shows `docments` for any function or method.  

In [3]:
def _non_empty_keys(d:dict): return L([k for k,v in d.items() if v != inspect._empty])
def _bold(s): return f'**{s}**' if s.strip() else s

In [4]:
#|export
def _maybe_nm(o): 
    if (o == inspect._empty): return ''
    else: return o.__name__ if hasattr(o, '__name__') else str(o)

In [5]:
test_eq(_maybe_nm(list), 'list')
test_eq(_maybe_nm('fastai'), 'fastai')

In [6]:
#|export
def _list2row(l:list): return '| '+' | '.join([_maybe_nm(o) for o in l]) + ' |'

In [7]:
#|hide
test_eq(_list2row(['Hamel', 'Jeremy']), '| Hamel | Jeremy |')
test_eq(_list2row([inspect._empty, bool, 'foo']), '|  | bool | foo |')

In [8]:
#|export
class DocmentTbl:
    # this is the column order we want these items to appear
    _map = OrderedDict({'anno':'Type', 'default':'Default', 'docment':'Details'})
    
    def __init__(self, obj, verbose=True, returns=True):
        "Compute the docment table string"
        self.verbose = verbose
        self.returns = False if isdataclass(obj) else returns
        self.params = L(inspect.signature(obj).parameters.keys())
        _dm = docments(obj, full=True, returns=returns)
        if 'self' in _dm: del _dm['self']
        for d in _dm.values(): d['docment'] = ifnone(d['docment'], inspect._empty)
        self.dm = _dm
    
    @property
    def _columns(self):
        "Compute the set of fields that have at least one non-empty value so we don't show tables empty columns"
        cols = set(flatten(L(self.dm.values()).filter().map(_non_empty_keys)))
        candidates = self._map if self.verbose else {'docment': 'Details'}
        return OrderedDict({k:v for k,v in candidates.items() if k in cols})
    
    @property
    def has_docment(self): return self._columns and self._row_list

    @property
    def has_return(self): return self.returns and bool(_non_empty_keys(self.dm.get('return', {})))
    
    def _row(self, nm, props): 
        "unpack data for single row to correspond with column names."
        return [nm] + [props[c] for c in self._columns]
    
    @property
    def _row_list(self):
        "unpack data for all rows."
        ordered_params = [(p, self.dm[p]) for p in self.params if p != 'self']
        return L([self._row(nm, props) for nm,props in ordered_params])
    
    @property
    def _hdr_list(self): return ['  '] + [_bold(l) for l in L(self._columns.values())]

    @property
    def hdr_str(self):
        "The markdown string for the header portion of the table"
        md = _list2row(self._hdr_list)
        return md + '\n' + _list2row(['-' * len(l) for l in self._hdr_list])
    
    @property
    def params_str(self): 
        "The markdown string for the parameters portion of the table."
        return '\n'.join(self._row_list.map(_list2row))
    
    @property
    def return_str(self):
        "The markdown string for the returns portion of the table."
        return _list2row(['**Returns**']+[_bold(_maybe_nm(self.dm['return'][c])) for c in self._columns])
    
    def __str__(self):
        if not self.has_docment: return ''
        _tbl = [self.hdr_str, self.params_str]
        if self.has_return: _tbl.append(self.return_str)
        return '\n'.join(_tbl)
    
    def __eq__(self,other): return self.__str__() == str(other).strip()

    def _repr_markdown_(self): return self.__str__()

`DocmentTbl` can render a markdown table showing `docments` if appropriate.  This is an example of how a `docments` table will render for a function:

In [9]:
def _f(a,      # description of param a 
       b=True, # description of param b
       c:str=None
       ) -> int: ...

_dm = DocmentTbl(_f)
_dm

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| a |  |  | description of param a |
| b | bool | True | description of param b |
| c | str | None |  |
| **Returns** | **int** |  |  |

In [10]:
#|hide
_exp_res="""
|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| a |  |  | description of param a |
| b | bool | True | description of param b |
| c | str | None |  |
| **Returns** | **int** |  |  |
"""

test_eq(_dm, _exp_res)

If one column in the table has no information, for example because there are no `docments`, that column will not be shown.  In the below example, the **Details** column, will not be shown.  Additionally, if the return of the function is not annotated the **Returns** row will not be rendered:

In [11]:
def _f(a, 
        b=True,
        c:str=None
       ): ...

_dm2 = DocmentTbl(_f)
_dm2

|    | **Type** | **Default** |
| -- | -------- | ----------- |
| a |  |  |
| b | bool | True |
| c | str | None |

In [12]:
_dm2.has_return

False

In [13]:
#|hide
_exp_res2 = """
|    | **Type** | **Default** |
| -- | -------- | ----------- |
| a |  |  |
| b | bool | True |
| c | str | None |
"""

test_eq(_dm2, _exp_res2)

`DocmentTbl` also works on classes.  By default, the `__init__` will be rendered:

In [14]:
class _Test:
    def __init__(self, 
                 a,      # description of param a 
                 b=True, # description of param b
                 c:str=None):
        ...
        
    def foo(self, 
            c:int,      # description of param c
            d=True, # description of param d
           ):
        ...

In [15]:
DocmentTbl(_Test)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| a |  |  | description of param a |
| b | bool | True | description of param b |
| c | str | None |  |

You can also pass a method to be rendered as well:

In [16]:
DocmentTbl(_Test.foo)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | int |  | description of param c |
| d | bool | True | description of param d |

In [17]:
#|hide
_exp_res3 = """
|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | int |  | description of param c |
| d | bool | True | description of param d |
"""

test_eq(DocmentTbl(_Test.foo), _exp_res3)

Dataclasses can also be rendered: 

In [18]:
@dataclass
class _TestClass:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self, tax=1.14) -> float:
        return self.unit_price * self.quantity_on_hand * tax

In [19]:
DocmentTbl(_TestClass)

|    | **Type** | **Default** |
| -- | -------- | ----------- |
| name | str |  |
| unit_price | float |  |
| quantity_on_hand | int | 0 |

In [20]:
#|hide
test_eq(DocmentTbl(_TestClass),
"""
|    | **Type** | **Default** |
| -- | -------- | ----------- |
| name | str |  |
| unit_price | float |  |
| quantity_on_hand | int | 0 |
""")

## Show Complete Documentation For An Object
Render the signature as well as the `docments` to show complete documentation for an object.

In [21]:
#|export
def get_name(obj):
    "Get the name of `obj`"
    if hasattr(obj, '__name__'):       return obj.__name__
    elif getattr(obj, '_name', False): return obj._name
    elif hasattr(obj,'__origin__'):    return str(obj.__origin__).split('.')[-1] #for types
    elif type(obj)==property:          return _get_property_name(obj)
    else:                              return str(obj).split('.')[-1]

In [22]:
#|export
def qual_name(obj):
    "Get the qualified name of `obj`"
    if hasattr(obj,'__qualname__'): return obj.__qualname__
    if inspect.ismethod(obj):       return f"{get_name(obj.__self__)}.{get_name(fn)}"
    return get_name(obj)

In [23]:
#|export
class ShowDocRenderer:
    def __init__(self, sym, disp:bool=True):
        "Show documentation for `sym`"
        store_attr()
        self.nm = qual_name(sym)
        self.isfunc = inspect.isfunction(sym)
        self.sig = inspect.signature(sym)
        self.docs = docstring(sym)
        self.dm = DocmentTbl(sym)

In [24]:
#|export
class BasicMarkdownRenderer(ShowDocRenderer):
    def _repr_markdown_(self):
        doc = '---\n\n'
        if self.isfunc: doc += '#'
        doc += f'### {self.nm}\n\n> **`{self.nm}`**` {self.sig}`'
        if self.docs: doc += f"\n\n{self.docs}"
        return doc

In [25]:
#|export
def show_doc(sym, disp=True, renderer=None):
    if renderer is None: renderer = get_config().get('renderer', None)
    if renderer is None: renderer=BasicMarkdownRenderer
    elif isinstance(renderer,str):
        p,m = renderer.rsplit('.', 1)
        renderer = getattr(import_module(p), m)
    return renderer(sym or show_doc, disp=disp)

You can use `show_doc` to document apis of functions, classes or methods:

In [26]:
def f(x:int=1):
    "func docstring"
    ...

show_doc(f)

---

#### f

> **`f`**` (x: int = 1)`

func docstring

In [27]:
class Foo:
    def __init__(d:str,e:int):
        "This is the docstring for the __init__ method"
        ...

show_doc(Foo)

---

### Foo

> **`Foo`**` (e: int)`

This is the docstring for the __init__ method

In [28]:
class Foo:
    def a_method(a:list,b:dict,c):
        "This is a method"
        ...

show_doc(Foo.a_method)

---

#### Foo.a_method

> **`Foo.a_method`**` (a: list, b: dict, c)`

This is a method

In [29]:
#|export
class BasicHtmlRenderer(ShowDocRenderer):
    def _repr_html_(self):
        doc = '<hr/>\n'
        lvl = 4 if self.isfunc else 3
        doc += f'<h{lvl}>{self.nm}</h{lvl}>\n<blockquote><code>{self.nm}{self.sig}</code></blockquote>'
        if self.docs: doc += f"<p>{self.docs}</p>"
        return doc

In [30]:
class F:
    "class docstring"
    def __init__(self, x:int=1): ...

show_doc(F, renderer=BasicHtmlRenderer)

In [31]:
#|export
def showdoc_nm(tree):
    "Get the fully qualified name for showdoc."
    return ifnone(get_patch_name(tree), tree.name)

In [32]:
#|hide
import ast
code="""
@bar
@patch
@foo
def a_method(self:Foo, a:list,b:dict,c):
    "This is a method"
    ...
"""

code2="""
@bar
@foo
def a_method(self:Foo, a:list,b:dict,c):
    "This is a method"
    ...
"""

_tree = ast.parse(code).body[0]
test_eq(showdoc_nm(_tree), 'Foo.a_method')

_tree2 = ast.parse(code2).body[0]
test_eq(showdoc_nm(_tree2), 'a_method')

## Export -

In [33]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()